In [1]:
import numpy as np
import pandas as pd

In [2]:
o_data_dir = '/home/artemy/multimodal_proj/data/AE_predictions/'

multi_gex_file = o_data_dir + "multi_gex_pred.npy"
cite_adt_file = o_data_dir + "cite_adt_pred.npy"

In [3]:
gex_predicted = np.load(multi_gex_file)
cite_predicted = np.load(cite_adt_file)

In [4]:
competion_dir = '/home/artemy/multimodal_proj/data/competition/'
eval_form = pd.read_csv(competion_dir + 'evaluation_ids.csv')




In [5]:
competion_dir = '/home/artemy/multimodal_proj/data/competition/'

cell_labels_cite = pd.read_hdf(competion_dir + 'test_cite_inputs.h5').index.tolist()
cell_labels_multi = pd.read_hdf(competion_dir + 'test_multi_inputs.h5').index.tolist()

In [15]:
len(set(cell_labels_multi)& set(eval_form['cell_id']))

16780

In [6]:
adt_features = pd.read_hdf(competion_dir + 'train_cite_targets.h5').columns.tolist()
gex_features = pd.read_hdf(competion_dir + 'train_multi_targets.h5').columns.tolist()

In [7]:
adt_di = dict(zip(adt_features, range(len(adt_features))))
gex_di = dict(zip(gex_features, range(len(gex_features))))

In [8]:
eval_form['idx'] = eval_form['gene_id'].apply(lambda x: adt_di[x] if x in adt_di else gex_di[x])

In [9]:
eval_form_grouped = eval_form.groupby('cell_id', sort=False)

In [10]:
outputs = []
for cell, df in eval_form_grouped:
    if cell in cell_labels_cite:
        cell_idx = cell_labels_cite.index(cell)
        outputs.append(cite_predicted[cell_idx, df.idx])
    else:
        cell_idx = cell_labels_multi.index(cell)
        outputs.append(gex_predicted[cell_idx, df.idx])
        
concat_outputs = np.concatenate(outputs)

In [11]:
submission = pd.read_csv(competion_dir + 'sample_submission.csv', index_col=0)
submission['target'] = concat_outputs
submission.to_csv(o_data_dir+'submission.csv')